<a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"><img src = "https://cdn.torontolife.com/wp-content/uploads/2016/10/toronto-skyline-803x603-1476458932.jpg" width = 140, align = "center"></a>

# Clustering Neighborhoods in Toronto
### Autor: Patrick Franco Alves

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Install the required packages</a>

2. <a href="#item2">BeautifulSoup to scrap the data</a>

3. <a href="#item3">Storing in pandas dataset</a>

4. <a href="#item4">Map of Toronto using amazing Folium</a>

</font>
</div>

## 1.  Install the required packages

In [11]:
#!conda install -c anaconda bs4 --yes
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.3            |           py36_0          60 KB  anaconda
    beautifulsoup4-4.8.0       |           py36_0         147 KB  anaconda
    ------------------------------------------------------------
                                           Total:         207 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.3-py36_0 anaconda

The following packages will be UPDATED:

    beautifulsoup4: 4.6.3-py37_0          --> 4.8.0-py36_0 anaconda


soupsieve-1.9.3      | 60 KB     | 

In [4]:
!conda install -c anaconda lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    certifi-2019.9.11          |           py36_0         154 KB  anaconda
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB  anaconda
    ------------------------------------------------------------
                                           Total:         6.7 MB

The following packages will be UPDATED:

    certifi: 2019.6.16-py36_1     conda-forge --> 2019.9.11-py36_0     anaconda
    lxml:    4.2.5-py37hefd8a0e_0             --> 

In [ ]:
## 2.  BeautifulSoup to scrap the data

In [37]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

## 2. Uses BeautifulSoup to scrap the data.

In [22]:
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_text, "html5lib")


In [23]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')


## 3.  Storing in pandas dataset

In [24]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

In [25]:
df.head(15)

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


In [43]:
df[df.Borough != 'Not assigned'].head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [46]:
df[df.Borough != 'Not assigned'].groupby('Neighbourhood').count()

,PostalCode,Borough
Neighbourhood,,
Adelaide,1,1
Agincourt,1,1
Agincourt North,1,1
Albion Gardens,1,1
Alderwood,1,1
...,...,...
Woodbine Heights,1,1
York Mills,1,1
York Mills West,1,1


In [47]:
df2 = df[df.Borough != 'Not assigned']

In [49]:
print("Before deleting Not assigned rows",df.shape)
print("After deleting Not assigned rows",df2.shape)

Before deleting Not assigned rows (288, 3)
After deleting Not assigned rows (211, 3)


##### Use geopy to get the latitude and longitude of Toronto.

In [13]:
!conda install -c conda-forge geopy --yes 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         238 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

In [14]:
from geopy.geocoders import Nominatim 

In [15]:
# convert an address into latitude and longitude values
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Latitude and longitude of Toronto: {}, {}.'.format(latitude, longitude))

Latitude and longitude of Toronto: 43.653963, -79.387207.


## 4. Map of Toronto using folium

In [17]:
import folium

In [28]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=14)

In [31]:
#map_newyork

In [32]:
!conda install -c conda-forge geocoder --yes 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    future-0.18.0              |           py36_0         714 KB  conda-forge
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         834 KB

The following NEW packages will be INSTALLED:

    future:   0.18.0-py36_0 conda

In [41]:
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [39]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')


Data downloaded!


In [40]:
#http://cocl.us/Geospatial_data
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
#merge
geo_df.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [52]:
df2.columns

Index(['PostalCode', 'Borough', 'Neighbourhood'], dtype='object')

In [53]:

neighborhoods = df2.merge(geo_df, left_on='PostalCode', right_on='Postal Code')


In [54]:
neighborhoods.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763


In [57]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork